In [1]:
from analysis.sim_import import parse_trips_file, parse_legs_file
from analysis.utils import load_df_to_database, load_db_parameters

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
trips = 'Y:/stuttgart-v1.0/05_stuttgart-v1.0_scenariotest/output/output-m5_2/m5_2.output_trips.csv.gz'
db_parameter = 'C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/03_analysis/01_calibration/db_param_10pct.json'
db_parameter = load_db_parameters(db_parameter)

2021-02-25 09:46:30,680 - INFO: Fetch db_parameter...


In [4]:
gdf = parse_trips_file(trips)

2021-02-25 09:46:35,876 - INFO: Parse trips file...
2021-02-25 09:46:51,181 - INFO: Build trip geometries...
2021-02-25 09:46:52,373 - INFO: Further data manipulations...
2021-02-25 09:46:54,614 - INFO: Trip table manipulation finished...


In [5]:
gdf.head(3)

,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,matsim_raw_main_mode,longest_distance_mode,...,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,geometry,matsim_cal_main_mode,arr_time,trip_speed,beeline_speed
0,68775050_423,1,68775050_423_1,56700,1057,0,13825,10483,ride,ride,...,76169649,474116.241443,5.461252e+06,NaN,NaN,"LINESTRING (466770.780 5468732.055, 474116.241...",ride,57757,47.086093,35.703690
1,68775050_423,2,68775050_423_2,62400,815,0,11474,10191,ride,ride,...,83041002,468191.397465,5.469544e+06,NaN,NaN,"LINESTRING (474116.241 5461251.768, 468191.397...",ride,63215,50.682699,45.015460
2,68775050_423,3,68775050_423_3,65700,7321,1704,110830,81415,pt_with_bike_used,pt,...,119912662,514970.054612,5.402909e+06,509649,8000096.2,"LINESTRING (468191.397 5469544.333, 514970.055...",pt,74725,44.209197,32.475789


In [10]:
gdf.loc[1144]

person                                                        70180025_1465
trip_number                                                               1
trip_id                                                     70180025_1465_1
dep_time                                                              30120
trav_time                                                              2830
wait_time                                                               338
traveled_distance                                                     11287
euclidean_distance                                                     7564
matsim_raw_main_mode                                                     pt
longest_distance_mode                                                    pt
modes                                                     walk-walk-pt-walk
start_activity_type                                            home_36600.0
end_activity_type                                              work_15000.0
start_facili

In [6]:
sample = gdf.sample(n=3)
sample

,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,matsim_raw_main_mode,longest_distance_mode,...,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,geometry,matsim_cal_main_mode,arr_time,trip_speed,beeline_speed
26640,71229095_364,2,71229095_364_2,46200,1955,109,6080,3938,pt,pt,...,3926967,500525.112784,5.404658e+06,555537.1,940406.4,"LINESTRING (504429.627 5405174.754, 500525.113...",pt,48264,10.604651,6.868605
22138,71063005_591,6,71063005_591_6,80220,340,0,360,282,walk,walk,...,105886177,512750.374126,5.402706e+06,NaN,NaN,"LINESTRING (513032.510 5402712.593, 512750.374...",walk,80560,3.811765,2.985882
32746,71638040_862,2,71638040_862_2,26093,3370,183,15356,11833,pt_with_bike_used,pt,...,111067461,514252.541708,5.403982e+06,562151.3,559453.1,"LINESTRING (516772.404 5415544.217, 514252.542...",pt,29646,15.559133,11.989530


In [7]:
table_name = 'sim_trips_raw'
table_schema = 'matsim_output'

DATA_METADATA = {
    'title': 'Trips',
    'description': 'Trip table',
    'source_name': 'Senozon Input',
    'source_url': 'Nan',
    'source_year': '2020',
    'source_download_date': 'Nan',
}

In [8]:
load_df_to_database(
    df=sample,
    update_mode='append',
    db_parameter=db_parameter,
    schema=table_schema,
    table_name=table_name,
    meta_data=DATA_METADATA,
    geom_cols={'geometry': 'LINESTRING'})

2021-02-25 09:47:15,179 - INFO: Import data to database...
2021-02-25 09:47:15,179 - INFO: Checking meta data inputs
2021-02-25 09:47:15,179 - INFO: Found GeoDataFrame
2021-02-25 09:47:15,183 - INFO: Checking for EPSG 25832
2021-02-25 09:47:15,183 - INFO: Writing wkts of geometries
C:\Users\david\.conda\envs\mynewenv\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
2021-02-25 09:47:15,187 - INFO: Uploading to database
2021-02-25 09:47:15,296 - INFO: Table metadata: 
        COMMENT ON TABLE matsim_output.sim_trips_raw IS
        '
        ####################
        # Meta information #
        ####################
            title: Trips,
            description: Trip table,
            source_name: Senozon Input,
            source_url: Nan,
            source_year: 2020,
            source_download_date: Nan'
        
2021-02-25 09:47:15,296 - INFO: Writing metadat

In [1]:
import pandas as pd

In [4]:
calib = 'C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/05_CalibrationData/Calibration_Data_v2.xlsx'
table = pd.read_excel(calib, sheet_name='05_OEVSegmente', skipfooter=10)

In [5]:
table

,oev_segment,sgmt_rel
0,oev_Bus,32.871537
1,oev_Regio_DB,8.060453
2,oev_Sbahn,27.141058
3,oev_SSB,31.926952
